# pystac: Working with catalogs

##### Example catalog structrure:
```
example-catalog/
├── catalog.json
└── eo
    ├── catalog.json
    ├── landsat-8-l1
    │   ├── catalog.json
    │   └── item.json
    └── sentinel-2-l1c
        ├── catalog.json
        └── sentinel-2a
            ├── catalog.json
            └── item.json
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from os.path import dirname
import pprint

from pystac import Catalog, Collection, Item

In [4]:
pp = pprint.PrettyPrinter(indent=4)

### Working with existing catalogs

In [5]:
# open a root catalog
cat = Catalog.from_file('example-catalog/catalog.json')

In [6]:
# get first (and only in this case) sub-catalog
subcat = cat.get_children()[0]

/home/simon/files/rv/stac/pystac/tutorials/example-catalog/eo/catalog.json


In [15]:
# print some IDs
print("Root Catalog: ", cat.id)
print("Sub Catalog: ", subcat.id)
print("Sub Catalog parent: ", subcat.get_parent().id)

# iterate through child catalogs of the sub-catalog
print("Sub Catalog children:")
for child in subcat.get_children():
    print('    ', child.id)

Root Catalog:  stac
Sub Catalog:  stac-eo
Sub Catalog parent:  stac
Sub Catalog children:
     sentinel-2-l1c
     landsat-8-l1


In [ ]:
all_items = cat.get_all_items()

print(all_items)

In [ ]:
print('\n**Items**')
for i in cat.items():
    print(i.id)

### Creating new catalogs

In [ ]:
# create a Catalog object with JSON
cat_json = {
    "id": "mycat",
    "description": "My shiny new STAC catalog",
    "href": "my_cat"
}
mycat = Catalog(**cat_json)

In [ ]:
mycat.describe()

### Adding catalogs to catalogs

In [ ]:
# add a new catalog to a root catalog
kitten_json = {
    "id": "mykitten",
    "description": "A child catalog of my shiny new STAC catalog"
}

kitten = Catalog(**kitten_json)

In [ ]:
mycat.add_child(kitten)

In [ ]:
mycat.describe()

### Adding collections to catalogs

In [ ]:
# open the Landsat collection
collection = Collection.from_file('example-catalog/eo/landsat-8-l1/catalog.json')
print('Collection name: ', collection.id)

In [ ]:
collection.describe()

In [ ]:
collection.links

In [ ]:
# add it to the child catalog created above
kitten.add_child(collection)
# print('Collection filename: ', collection.filename)

In [ ]:
mycat.describe()

### Adding items to collection

In [ ]:
# open a Landsat item
item = Item.from_file('example-catalog/eo/sentinel-2-l1c/sentinel-2a/item.json')
print('Item name: ', item.id)

In [ ]:
collection.describe()

In [ ]:
# add it to the collection created above
collection.add_item(item)
# print('Item filename: ', item.filename)

# print('\n**Item links**')
# pp.pprint(item._data['links'])

In [ ]:
mycat.set_uris_from_root('/home/simon/files/rv/stac/pystac-example/')

In [ ]:
mycat.describe

In [ ]:
mycat.save()